In [1]:
import pandas as pd

In [2]:
train = pd.read_csv("labeledTrainData.tsv", sep='\t')
test = pd.read_csv("testData.tsv", sep='\t')
sample = pd.read_csv("sample.csv")

In [3]:
print train.head(2)
print test.head(2)
print sample.head(2)
print train.count()
print test.count()

       id  sentiment                                             review
0  5814_8          1  With all this stuff going down at the moment w...
1  2381_9          1  \The Classic War of the Worlds\" by Timothy Hi...
         id                                             review
0  12311_10  Naturally in a film who's main themes are of m...
1    8348_2  This movie is a disaster within a disaster fil...
         id  sentiment
0  12311_10          0
1    8348_2          0
id           25000
sentiment    25000
review       25000
dtype: int64
id        25000
review    25000
dtype: int64


In [46]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
import nltk

def review_to_wordlist(review, remove_stopwords=False):    
    # Function to convert a document to a sequence of words,    
    # optionally removing stop words.  Returns a list of words.    
    #    
    # 1. Remove HTML    
    review_text = BeautifulSoup(review).get_text()    
    #      
    # 2. Remove non-letters    
    review_text = re.sub("[^a-zA-Z]"," ", review_text)    
    #    
    # 3. Convert words to lower case and split them    
    words = review_text.lower().split()    
    #    
    # 4. Optionally remove stop words (false by default)    
    if remove_stopwords:        
        stops = set(stopwords.words("english"))        
        words = [w for w in words if not w in stops]    
        #    
        # 5. Return a list of words    
    return(words)

In [8]:
# nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [33]:
# from nltk import word_tokenize
tokenizer = nltk.data.load('nltk:tokenizers/punkt/english.pickle')

In [50]:
# Define a function to split a review into parsed sentences
def review_to_sentences(review, tokenizer, remove_stopwords=False):    
    # Function to split a review into parsed sentences. Returns a     
    # list of sentences, where each sentence is a list of words    
    #    
    # 1. Use the NLTK tokenizer to split the paragraph into sentences 
    raw_sentences = tokenizer.tokenize(review.strip().decode('utf-8'))
    
    #    
    # 2. Loop over each sentence    
    sentences = []    
    for raw_sentence in raw_sentences:        
        # If a sentence is empty, skip it        
        if len(raw_sentence) > 0:            
            # Otherwise, call review_to_wordlist to get a list of words            
            sentences.append(review_to_wordlist(raw_sentence, remove_stopwords))    
            #    
            # Return the list of sentences (each sentence is a list of words,    
            # so this returns a list of lists    
    return sentences

In [18]:
# unlabeled_train = pd.read_csv("unlabeledTrainData.tsv", sep='\t')

In [41]:
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> stopwords
      Unzipping corpora/stopwords.zip.

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

In [51]:
sentences = []  # Initialize an empty list of sentences

print "Parsing sentences from training set"
for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)

/home/ubuntu/anaconda/lib/python2.7/site-packages/bs4/__init__.py:182: UserWarning: "." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  '"%s" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.' % markup)
/home/ubuntu/anaconda/lib/python2.7/site-packages/bs4/__init__.py:189: UserWarning: "http://www.happierabroad.com" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)


Parsing sentences from training set


In [52]:
print len(sentences)
print sentences[0]

266885
[u'with', u'all', u'this', u'stuff', u'going', u'down', u'at', u'the', u'moment', u'with', u'mj', u'i', u've', u'started', u'listening', u'to', u'his', u'music', u'watching', u'the', u'odd', u'documentary', u'here', u'and', u'there', u'watched', u'the', u'wiz', u'and', u'watched', u'moonwalker', u'again']


In [53]:
from gensim.models import word2vec

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)from gensim.models import word2vec
print "Training model..."

model = word2vec.Word2Vec(sentences, workers=num_workers, size=num_features, min_count = min_word_count, window = context, sample = downsampling)

# If you don't plan to train the model any further, calling # init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context.pkl"
model.save(model_name)

Training model...


Using gpu device 0: GRID K520


In [54]:
print model.doesnt_match("man woman child kitchen".split())
print model.doesnt_match("france england germany berlin".split())
print model.doesnt_match("paris berlin london austria".split())

kitchen
berlin
paris


In [55]:
model.most_similar("man")

[(u'woman', 0.6002646684646606),
 (u'guy', 0.4918012022972107),
 (u'boy', 0.4908530116081238),
 (u'lady', 0.48227813839912415),
 (u'named', 0.4763108491897583),
 (u'girl', 0.45790719985961914),
 (u'doctor', 0.4178612232208252),
 (u'soldier', 0.4119168519973755),
 (u'son', 0.40765318274497986),
 (u'businessman', 0.4072432518005371)]

In [56]:
model.most_similar("movie")

[(u'film', 0.7240103483200073),
 (u'it', 0.49680161476135254),
 (u'movies', 0.49463677406311035),
 (u'show', 0.428367555141449),
 (u'flick', 0.4108177423477173),
 (u'really', 0.3872818946838379),
 (u'thing', 0.38200855255126953),
 (u'just', 0.37946921586990356),
 (u'sequel', 0.37930771708488464),
 (u'because', 0.3668331503868103)]

In [57]:
model.most_similar("kill")

[(u'killing', 0.5700429081916809),
 (u'killed', 0.5425001382827759),
 (u'kills', 0.5362086296081543),
 (u'protect', 0.5012085437774658),
 (u'decide', 0.49004626274108887),
 (u'shoot', 0.48850518465042114),
 (u'jail', 0.4791141152381897),
 (u'die', 0.47435376048088074),
 (u'drive', 0.4721546173095703),
 (u'steal', 0.4617525339126587)]

In [58]:
type(model.syn0)

numpy.ndarray

In [59]:
model.syn0.shape

(8306, 300)

In [60]:
model.syn0

array([[ 0.11301138,  0.09196355,  0.02227202, ...,  0.0932686 ,
         0.07330729,  0.05409562],
       [-0.10085322,  0.00798355,  0.06586684, ...,  0.07878271,
         0.07288806,  0.05811012],
       [-0.1237859 ,  0.02547193,  0.08878306, ...,  0.11291546,
         0.07088824,  0.13749835],
       ..., 
       [ 0.08134036,  0.19218518, -0.04442903, ...,  0.09148403,
        -0.01917326, -0.02962955],
       [ 0.0599947 ,  0.16439657, -0.03100142, ...,  0.12783648,
        -0.04256465,  0.12394594],
       [ 0.03108146,  0.02801377,  0.05658953, ...,  0.07385855,
        -0.03732267,  0.00617132]], dtype=float32)

In [61]:
print model["flower"].shape
model["flower"]

(300,)


array([  9.19017661e-03,   6.54114336e-02,   1.33776793e-03,
        -1.81574468e-02,   4.02462948e-03,  -5.59470057e-03,
        -1.72537216e-03,  -7.38028288e-02,   6.71816012e-03,
        -3.69991884e-02,   4.19161059e-02,  -2.58125439e-02,
         8.52258317e-03,   9.11382437e-02,  -1.23168685e-01,
         1.05112046e-01,  -7.69874081e-02,  -7.29553998e-02,
        -4.31609750e-02,  -9.53138396e-02,  -2.66510025e-02,
         8.86965543e-02,   6.93313628e-02,  -1.32601196e-02,
         4.21601236e-02,   5.42824268e-02,   4.79070656e-02,
        -8.12596604e-02,  -8.10332149e-02,  -2.31435173e-03,
        -3.37034203e-02,  -2.09816005e-02,   3.05269510e-02,
         4.79896180e-03,  -1.06143104e-02,   4.57150824e-02,
         8.17497000e-02,   2.96192449e-02,  -5.13435528e-03,
        -3.79809141e-02,  -7.90826380e-02,  -6.15077466e-02,
        -1.99032500e-02,  -1.64032474e-01,  -3.05936057e-02,
        -7.30207562e-02,   9.87990350e-02,   2.07166597e-02,
        -3.08413655e-02,

In [64]:
import numpy as np

In [65]:
def makeFeatureVec(words, model, num_features):    
    # Function to average all of the word vectors in a given    
    # paragraph    
    #    
    # Pre-initialize an empty numpy array (for speed)    
    featureVec = np.zeros((num_features,),dtype="float32")    
    #    
    nwords = 0.    
    #     
    # Index2word is a list that contains the names of the words in     
    # the model's vocabulary. Convert it to a set, for speed     
    index2word_set = set(model.index2word)    
    #    
    # Loop over each word in the review and, if it is in the model's    
    # vocaublary, add its feature vector to the total    
    for word in words:        
        if word in index2word_set:             
            nwords = nwords + 1.            
            featureVec = np.add(featureVec,model[word])    
    #     
    # Divide the result by the number of words to get the average    
    featureVec = np.divide(featureVec,nwords)    
    return featureVec

def getAvgFeatureVecs(reviews, model, num_features):    
    # Given a set of reviews (each one a list of words), calculate     
    # the average feature vector for each one and return a 2D numpy array     
    #     
    # Initialize a counter    
    counter = 0.    
    #     
    # Preallocate a 2D numpy array, for speed    
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")    
    #     
    # Loop through the reviews    
    for review in reviews:       
        #       
        # Print a status message every 1000th review       
        if counter%1000. == 0.:           
            print "Review %d of %d" % (counter, len(reviews))       
            #        
            # Call the function (defined above) that makes average feature vectors       
        reviewFeatureVecs[counter] = makeFeatureVec(review, model, num_features)       
        #       
        # Increment the counter       
        counter = counter + 1.    
    return reviewFeatureVecs

In [66]:
# ****************************************************************
# Calculate average feature vectors for training and testing sets,
# using the functions we defined above. Notice that we now use stop word
# removal.

clean_train_reviews = []
for review in train["review"]:    
    clean_train_reviews.append( review_to_wordlist( review, remove_stopwords=True ))
trainDataVecs = getAvgFeatureVecs( clean_train_reviews, model, num_features )

print "Creating average feature vecs for test reviews"
clean_test_reviews = []
for review in test["review"]:    
    clean_test_reviews.append( review_to_wordlist( review, remove_stopwords=True ))
testDataVecs = getAvgFeatureVecs( clean_test_reviews, model, num_features )

Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000
Creating average feature vecs for test reviews
Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 1900

In [67]:
print trainDataVecs.shape
print testDataVecs.shape

(25000, 300)
(25000, 300)


In [68]:
# LB : 0.81744 / 482nd/510

# Fit a random forest to the training data, using 100 trees
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators = 100)

print "Fitting a random forest to labeled training data..."
clf.fit(trainDataVecs, train["sentiment"])

# Test & extract results 
result = clf.predict(testDataVecs)

# Write the test results 
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )
output.to_csv("submission.csv", index=False, quoting=3)

Fitting a random forest to labeled training data...


In [74]:
# LB : 0.73900
from sklearn.naive_bayes import GaussianNB

clf = GaussianNB()

clf.fit(trainDataVecs, train["sentiment"])
result = clf.predict(testDataVecs)

# Write the test results 
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )
output.to_csv("submission.csv", index=False, quoting=3)

In [75]:
from sklearn.svm import SVC
from sklearn.grid_search import GridSearchCV

parameters = {'C' : [1, 10, 10, 1000]}
model = GridSearchCV(SVC(cache_size=2000, kernel="rbf"), parameters, n_jobs=4, verbose=1)

# Fit Grid Search Model
model.fit(trainDataVecs, train["sentiment"])

Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=4)]: Done   1 jobs       | elapsed:  5.1min
[Parallel(n_jobs=4)]: Done   6 out of  12 | elapsed:  9.0min remaining:  9.0min
[Parallel(n_jobs=4)]: Done  12 out of  12 | elapsed: 11.8min finished


Best score: 0.858
Best parameters set:


NameError: name 'param_grid' is not defined

In [80]:
print("Best score: %0.3f" % model.best_score_)
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(best_parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

clf = model.best_estimator_
    
clf.fit(trainDataVecs, train["sentiment"])
result = clf.predict(testDataVecs)

# LB : 0.85348 / 259nd/510

# Write the test results 
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )
output.to_csv("submission.csv", index=False, quoting=3)    

Best score: 0.858
Best parameters set:
	C: 1000
	cache_size: 2000
	class_weight: None
	coef0: 0.0
	degree: 3
	gamma: 0.0
	kernel: 'rbf'
	max_iter: -1
	probability: False
	random_state: None
	shrinking: True
	tol: 0.001
	verbose: False


In [81]:
parameters = {'C' : [1000, 1500, 2500, 3000, 3500, 4000, 4500, 5000]}
model = GridSearchCV(SVC(cache_size=2000, kernel="rbf"), parameters, n_jobs=4, verbose=1)

# Fit Grid Search Model
model.fit(trainDataVecs, train["sentiment"])

print("Best score: %0.3f" % model.best_score_)
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(best_parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))
    
clf = model.best_estimator_
    
clf.fit(trainDataVecs, train["sentiment"])
result = clf.predict(testDataVecs)

# LB : 0.85600 / 250nd/510

# Write the test results 
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )
output.to_csv("submission.csv", index=False, quoting=3)    

Fitting 3 folds for each of 8 candidates, totalling 24 fits


[Parallel(n_jobs=4)]: Done   1 jobs       | elapsed:  2.1min
[Parallel(n_jobs=4)]: Done  18 out of  24 | elapsed:  9.7min remaining:  3.2min
[Parallel(n_jobs=4)]: Done  24 out of  24 | elapsed: 11.7min finished


Best score: 0.860
Best parameters set:
	C: 5000
	cache_size: 2000
	class_weight: None
	coef0: 0.0
	degree: 3
	gamma: 0.0
	kernel: 'rbf'
	max_iter: -1
	probability: False
	random_state: None
	shrinking: True
	tol: 0.001
	verbose: False


In [82]:
parameters = {'C' : [5000, 6000, 7000, 8000, 9000, 10000, 11000, 12000]}
model = GridSearchCV(SVC(cache_size=2000, kernel="rbf"), parameters, n_jobs=4, verbose=1)

# Fit Grid Search Model
model.fit(trainDataVecs, train["sentiment"])

print("Best score: %0.3f" % model.best_score_)
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(best_parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))
    
clf = model.best_estimator_
    
clf.fit(trainDataVecs, train["sentiment"])
result = clf.predict(testDataVecs)

# LB : 0.85684 248nd/510

# Write the test results 
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )
output.to_csv("submission.csv", index=False, quoting=3)    

Fitting 3 folds for each of 8 candidates, totalling 24 fits


[Parallel(n_jobs=4)]: Done   1 jobs       | elapsed:  1.8min
[Parallel(n_jobs=4)]: Done  18 out of  24 | elapsed:  9.3min remaining:  3.1min
[Parallel(n_jobs=4)]: Done  24 out of  24 | elapsed: 11.1min finished


Best score: 0.861
Best parameters set:
	C: 12000
	cache_size: 2000
	class_weight: None
	coef0: 0.0
	degree: 3
	gamma: 0.0
	kernel: 'rbf'
	max_iter: -1
	probability: False
	random_state: None
	shrinking: True
	tol: 0.001
	verbose: False


In [83]:
parameters = {'C' : [12000, 20000, 30000, 40000, 50000, 60000, 70000, 80000,
                    90000, 100000, 110000, 120000, 130000, 140000, 150000]}
model = GridSearchCV(SVC(cache_size=2000, kernel="rbf"), parameters, n_jobs=5, verbose=1)

# Fit Grid Search Model
model.fit(trainDataVecs, train["sentiment"])

print("Best score: %0.3f" % model.best_score_)
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(best_parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Fitting 3 folds for each of 15 candidates, totalling 45 fits


[Parallel(n_jobs=5)]: Done   1 jobs       | elapsed:  2.2min
[Parallel(n_jobs=5)]: Done  37 out of  45 | elapsed: 20.4min remaining:  4.4min
[Parallel(n_jobs=5)]: Done  45 out of  45 | elapsed: 23.7min finished


Best score: 0.861
Best parameters set:
	C: 120000
	cache_size: 2000
	class_weight: None
	coef0: 0.0
	degree: 3
	gamma: 0.0
	kernel: 'rbf'
	max_iter: -1
	probability: False
	random_state: None
	shrinking: True
	tol: 0.001
	verbose: False


In [84]:
parameters = {'gamma' : [0.1, 0.3, 0.5, 0.7, 0.9, 0.01, 0.03, 0.05, 0.07, 0.09,
                         0.001, 0.003, 0.005, 0.007, 0.009, 0.0001, 0.0003, 0.0005, 0.0007, 0.0009]}
model = GridSearchCV(SVC(cache_size=2000, kernel="rbf", C=12000), parameters, n_jobs=5, verbose=1)

# Fit Grid Search Model
model.fit(trainDataVecs, train["sentiment"])

print("Best score: %0.3f" % model.best_score_)
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(best_parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=5)]: Done   1 jobs       | elapsed:  4.0min
[Parallel(n_jobs=5)]: Done  50 jobs       | elapsed: 39.1min
[Parallel(n_jobs=5)]: Done  52 out of  60 | elapsed: 40.8min remaining:  6.3min
[Parallel(n_jobs=5)]: Done  60 out of  60 | elapsed: 43.3min finished


Best score: 0.863
Best parameters set:
	C: 12000
	cache_size: 2000
	class_weight: None
	coef0: 0.0
	degree: 3
	gamma: 0.09
	kernel: 'rbf'
	max_iter: -1
	probability: False
	random_state: None
	shrinking: True
	tol: 0.001
	verbose: False


In [85]:
parameters = {'gamma' : [0.085, 0.086, 0.087, 0.088, 0.089, 0.09, 0.091, 0.092, 0.093, 0.094, 0.095]}
model = GridSearchCV(SVC(cache_size=2000, kernel="rbf", C=12000), parameters, n_jobs=6, verbose=1)

# Fit Grid Search Model
model.fit(trainDataVecs, train["sentiment"])

print("Best score: %0.3f" % model.best_score_)
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(best_parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Fitting 3 folds for each of 11 candidates, totalling 33 fits


[Parallel(n_jobs=6)]: Done   1 jobs       | elapsed:  4.6min
[Parallel(n_jobs=6)]: Done  33 out of  33 | elapsed: 25.4min finished


Best score: 0.864
Best parameters set:
	C: 12000
	cache_size: 2000
	class_weight: None
	coef0: 0.0
	degree: 3
	gamma: 0.085
	kernel: 'rbf'
	max_iter: -1
	probability: False
	random_state: None
	shrinking: True
	tol: 0.001
	verbose: False


In [86]:
clf = model.best_estimator_
    
clf.fit(trainDataVecs, train["sentiment"])
result = clf.predict(testDataVecs)

# LB : 

# Write the test results 
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )
output.to_csv("submission.csv", index=False, quoting=3)    